In [2]:
import clearml
from tqdm import tqdm
from transformers import logging
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from util import read_jsonl, set_random_seed, fix_tokenizer
from trl import DPOConfig, DPOTrainer
import pandas as pd
from datasets import load_dataset
import os

отключаем wandb 

In [2]:
os.environ["WANDB_DISABLED"] = "true"

Чекпоинты с которых стартуем

In [3]:
["../Train/models/sft_clean_only_2", "../Train/models/full-sft/full-sft"]

['../Train/models/sft_clean_only_2', '../Train/models/full-sft/full-sft']

In [4]:
dataset = load_dataset("./data/all")

In [5]:
sft_checkpoint = "../Train/models/sft_clean_only_2"

In [6]:
training_args = DPOConfig(
    seed=42,
    data_seed=42,
    model_adapter_name="train",
    output_dir="./models/dpo_sft_clean_only",
    ref_adapter_name="reference",
    num_train_epochs=3,
    per_device_eval_batch_size=8,
    per_device_train_batch_size=8,
    eval_strategy="steps",
    eval_steps=100,
    save_steps=200,
    learning_rate=5e-5,
    save_total_limit=2,
    save_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    logging_steps=50
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [7]:
def train(args: DPOConfig,
          sft_checkpoint):
    logging.set_verbosity_info()

    model = AutoModelForSeq2SeqLM.from_pretrained(sft_checkpoint).to("cuda")
    model_ref = AutoModelForSeq2SeqLM.from_pretrained(sft_checkpoint).to("cuda")
    tokenizer = AutoTokenizer.from_pretrained(sft_checkpoint, do_lower_case=False, strip_accents=False)
    tokenizer = fix_tokenizer(tokenizer)

    dpo_trainer = DPOTrainer(
        model,
        model_ref,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        tokenizer=tokenizer,
        args=args
    )
    dpo_trainer.train()
    dpo_trainer.save_model()

In [8]:
task = clearml.Task.init(project_name='cv-generation', task_name='dpo-clean-only-test')
train(training_args, sft_checkpoint)
task.close()

ClearML Task: overwriting (reusing) task id=5702d94a055a4d7db83284b8500315a8
2024-06-21 05:49:36,436 - clearml.Repository Detection - WARNING - Could not read Jupyter Notebook: No module named 'nbconvert'
2024-06-21 05:49:36,453 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/4dfada0d47c54d358198351db8a35944/experiments/5702d94a055a4d7db83284b8500315a8/output/log


loading configuration file ../Train/models/sft_clean_only_2\config.json
Model config T5Config {
  "_name_or_path": "../Train/models/sft_clean_only_2",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "bos_token_id": 2,
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 2,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "max_length": 200,
  "model_type": "t5",
  "num_beams": 5,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.41.2",
  "use_cache": true,
  "vocab_size": 30000
}

load

Vocab size:  30000
PAD:  0 <pad>
BOS:  None None
EOS:  1 </s>
UNK:  2 <unk>
SEP:  None None


Map: 100%|██████████| 630/630 [00:00<00:00, 1799.46 examples/s]
***** Running training *****
  Num examples = 5,665
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2,127
  Number of trainable parameters = 244,309,248
Automatic ClearML logging enabled.
External ClearML Task has been connected.


2024-06-21 05:49:52,526 - clearml.Task - WARNING - Parameters must be of builtin type (Transformers/accelerator_config[AcceleratorConfig])


...

{'eval_loss': 0.22476530075073242, 'eval_runtime': 10.9485, 'eval_samples_per_second': 57.542, 'eval_steps_per_second': 7.216, 'eval_rewards/chosen': -3.591540813446045, 'eval_rewards/rejected': -9.646472930908203, 'eval_rewards/accuracies': 0.8987341523170471, 'eval_rewards/margins': 6.054932594299316, 'eval_logps/rejected': -209.5202178955078, 'eval_logps/chosen': -84.9247817993164, 'eval_logits/rejected': -15.45541763305664, 'eval_logits/chosen': -20.113487243652344, 'epoch': 0.06}


Model weights saved in ./models/dpo_sft_clean_only\checkpoint-40\model.safetensors
tokenizer config file saved in ./models/dpo_sft_clean_only\checkpoint-40\tokenizer_config.json
Special tokens file saved in ./models/dpo_sft_clean_only\checkpoint-40\special_tokens_map.json


KeyboardInterrupt: 

2024-06-21 05:52:57,096 - clearml.Task - WARNING - ### TASK STOPPED - USER ABORTED - STATUS CHANGED ###


: 